In [1]:
# Imports a parser from cogent
from cogent.parse.fasta import MinimalFastaParser as parse

In [2]:
# applies for the whole segment
nprocs = 10

In [10]:
# Checking out data file.
# This file was created using the QC_basic notebook.
!head data/finalQC_Pool1_ITS.fasta

>13C.NTH.D7.R4_Frac11_0
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGGCTCTGGC
>13C.PTH.D30.R4_Frac12_1
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGGCTCTGGC
>13C.PTH.D14.R4_Frac22_2
TTAAACTCATCTCAATTTATTACCTGTAACCCGCCTTAAACCGGGCACACCGGGATCGGGCCTGATAGACTTCACGGTTTGTCGGGTGGCAACCCAATAACCAAAACCTTTTTTAACTTTGTCCATTTCCCGGCTTCCGAAAGGTTGTCGGTTTTCCATCAAACACCATTACCGGTGGATGACTCGGCTCAGAC
>13X.NTH.D3.R4_Frac25_3
TTATCGAGTTTTGACTGGGTTGTAGCTGGCCTTCCGAGGCATGTGCACGCCCTGCTCATCCACTCTACACCTGTGCACTTACTGTGGGTATCAGATCGTGAAGCGTGCTCTTTTACCGGAGCTTGTGAAGCGTGTCTGTGCCTGCGTTTATCACAAACACTATAAAGTATCAGAATGTGTATTACGATGTAACGCATCTATATACAACTTTCAGCAACGGATCTCTTGGCTCTCGC
>13X.NTH.D7.R4_Frac25_4
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGAC

In [11]:
!sed 's/R4_/R4\./g' data/finalQC_Pool1_ITS.fasta > data/finalQC_Pool1_ITS_nound.fasta

In [14]:
!grep -c ">" data/finalQC_Pool1_ITS_nound.fasta

6884936


In [15]:
%%bash
# Running the seq separation on mothur instead

mothur "#unique.seqs(fasta=data/finalQC_Pool1_ITS_nound.fasta)" > /dev/null

In [18]:
# Making a dictionary of the names files, splitting it into the first (ID) and second (commas list of all seqs in it)
# Then it counts their lengths and saves it in the dictionary

counts = {}

with open("data/finalQC_Pool1_ITS_nound.names") as f:
    for line in f:
        seedID, seqIDs = line.split("\t")
        count = len(seqIDs.split(","))
        counts[seedID] = count        

In [23]:
# Adds the counts from this dictionary to our finalQC.unique file so it looks like a usearch file with "size=XXX"

with open("data/finalQC_Pool1_ITS.unique.usearch_names.fasta", "w") as f:
    for n, s in parse(open("data/finalQC_Pool1_ITS_nound.unique.fasta")):
        f.write(">%s;size=%s;\n%s\n"%(n,counts[n],s))  

In [24]:
!head data/finalQC_Pool1_ITS.unique.usearch_names.fasta

>13C.NTH.D7.R4.Frac11_0;size=1430816;
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGGCTCTGGC
>13C.PTH.D14.R4.Frac22_2;size=19;
TTAAACTCATCTCAATTTATTACCTGTAACCCGCCTTAAACCGGGCACACCGGGATCGGGCCTGATAGACTTCACGGTTTGTCGGGTGGCAACCCAATAACCAAAACCTTTTTTAACTTTGTCCATTTCCCGGCTTCCGAAAGGTTGTCGGTTTTCCATCAAACACCATTACCGGTGGATGACTCGGCTCAGAC
>13X.NTH.D3.R4.Frac25_3;size=9465;
TTATCGAGTTTTGACTGGGTTGTAGCTGGCCTTCCGAGGCATGTGCACGCCCTGCTCATCCACTCTACACCTGTGCACTTACTGTGGGTATCAGATCGTGAAGCGTGCTCTTTTACCGGAGCTTGTGAAGCGTGTCTGTGCCTGCGTTTATCACAAACACTATAAAGTATCAGAATGTGTATTACGATGTAACGCATCTATATACAACTTTCAGCAACGGATCTCTTGGCTCTCGC
>13C.PTH.D30.R4.Frac26_6;size=9457;
TTACCATAATACCAAAACCAAAAACACCTTACGAAGAGGAAACGTGAAGTTCGTGTATACGTCTTTGCGTCAGTACTCTTTGTACGTTTCGTGATACTTAACAACTATCATTATTCCTTTATGCTTCCTTAGCGATGTAAAAGCCTTTGGCTTTTGAAGTCATTGAAGATACAACACCATTAACCAGAGAACTCTGTCGAAATCTGTCAGTCATGACATGATTATCATATAATCAATGACCAAACAGTTTCAGCAAC

In [26]:
# Chuck looking to see how many Gb this file is.
!du -h data/finalQC_Pool1_ITS_nound.fasta

1.4G	data/finalQC_Pool1_ITS_nound.fasta


In [27]:
# Sequences are sorted by size
# Here the size of clusters - we are excluding the singletons here
# You would change minsize to 1 if you wanted to include singletons
# Or, you know, just not do this step.
# But you should just get rid of them.
!usearch -sortbysize data/finalQC_Pool1_ITS.unique.usearch_names.fasta -output data/finalQC_Pool1_ITS.unique_sorted.fasta -minsize 2

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:00 2.1Mb Reading data/finalQC_Pool1_ITS.unique.usearch_names.fasta, 313Mb
00:01 315Mb 1260765 (1.3M) seqs, min 50, avg 209, max 496nt
00:02 330Mb Getting sizes
00:03 340Mb Sorting 141290 sequences
00:03 341Mb  100.0% Writing data/finalQC_Pool1_ITS.unique_sorted.fasta


In [28]:
# Checking data
# You can see here, the first two sequences we saw above are now gone.
!head data/finalQC_Pool1_ITS.unique_sorted.fasta

>13C.NTH.D7.R4.Frac11_0;size=1430816;
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCC
GCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGG
CTCTGGC
>13X.NTH.D7.R4.Frac9_10;size=238696;
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTCACGCGCC
GCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGG
CTCTGGC
>13C.PTH.D30.R4.Frac18_36;size=147046;
TTATCAAACTTTTAAACCTAATAACAACCATAGAATAATCGTTTTTATCTTAATATATAATTTATAACACTTAAAATATC


In [29]:
# This is the clustering command.
# Default is 97% minimum ID.
# Not recommended to use more than 97%.
# Creates the centroids, or "seeds"
# Then you can take them out
!usearch -cluster_otus data/finalQC_Pool1_ITS.unique_sorted.fasta -otus data/otus_Pool1_ITS.fasta

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

01:42 196Mb  100.0% 23192 OTUs
                              
Input seqs  141290 (141.3k)
      OTUs  23192 (23.2k)
   Members  116456 (116.5k)
  Chimeras  1642
   Max mem  196Mb
      Time  01:42
Throughput  1385.2 seqs/sec.



In [30]:
# Making another file
# Figure this out (what is this?)
# This is a script (fasta_number.py) that replaces fasta names with XXX1, XXX2, etc.
# In our case, it is replacing the names with OTU.1, OTU.2, etc., and outputs it into a file called otusn.fasta
!/opt/bioinfo/edgar_python_scripts/fasta_number.py data/otus_Pool1_ITS.fasta OTU. > data/otusn_Pool1_ITS.fasta

In [31]:
!head data/otusn_Pool1_ITS.fasta

>OTU.1
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCC
GCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAaaaCTTTTGGCAACGGATCTCTTGG
CTCTGGC
>OTU.2
TTATCAAACTTTTAAACCTAATAACAACCATAGAATAATCGTTtttATCTTAATatataATTTATAACACTTAAAATATC
TTAAATGATTAAACCTTAAaaaaGTTTAAATGGAAaaaGCTTTTAACAATGGATACCTTGGTTCCTCT
>OTU.3
TTATTGAATAAACTTtttAAGTTtttttCATCTTCCCTGTTTATATTAAATACGTTCCTTCCATCGGACAGGTCTATTTA
GGACCCTCTAGCTTGGGCAAATGCTTTTGCTGGAGAGTTGCCGGTGGCATTTCTAAaaaTGAAAACTTTTGTTATATTAT


### Removing Chloroplast, Bacterial, and Archaeal sequences

In [32]:
# You do need to assign taxonomy in order to pull out the Euks., etc.
# I could cp these files from the server to wherever I need them.
# Input is your fasta file
# Output is a fasta with taxonomy assinged (still working with unique seqs)
# This didn't work first, because Chuck had to delete a "jobs" folder in the tmp directory.

!parallel_assign_taxonomy_uclust.py \
-r /home/chantal/RNASIP/data/databases/Silva_111_post/no_ambiguous_bases_files/97_Silva_111_rep_set_no_ambig.fasta \
-t /home/chantal/RNASIP/data/databases/Silva_111_post/taxonomy/Silva_111_taxa_map_full.txt \
-O 10 \
-i data/otusn_Pool1_ITS.fasta \
-o data/otusn_Pool1_ITS_tax

In [48]:
# This makes a file of what we want to remove
# Could change this to pull out different groups.
# These primers actually had good Archaeal targets - so, it would be okay to include them.
!egrep "Chloroplast|Bacteria|Archaea|mitochondria" \
data/otusn_Pool1_ITS_tax/otusn_Pool1_ITS_tax_assignments.txt \
| awk '{print $1}' > data/to_remove_tax.accnos

In [49]:
# wc is number of lines of the taxa that will be removed
!wc -l data/to_remove_tax.accnos

0 data/to_remove_tax.accnos


In [50]:
# Looking at what you're removing
!head data/to_remove_tax.accnos

In [51]:
%%bash
# Remove.seqs command will actually remove these taxa
mothur "#remove.seqs(fasta=data/otusn_Pool1_ITS.fasta, \
accnos=data/to_remove_tax.accnos)" #> /dev/null







mothur v.1.32.1
Last updated: 10/16/2013

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
pschloss@umich.edu
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

Type 'quit()' to exit program



mothur > remove.seqs(fasta=data/otusn_Pool1_ITS.fasta, accnos=data/to_remove_tax.accnos)
[ERROR]: data/to_remove_tax.accnos is blank, aborting.
You have no valid accnos file and accnos is required.
[ERROR]: did not complete remove.seqs.

mothur > quit()


### Mapping Reads

In [52]:
# Pulling out the sample identifier.
# It is adding a portion to the finalQC file that has the barcode label.
# Then we can use this later
# Now we will see how these reads map to the defined centroids (after removing EuK, etc.)
# Basically, we cut, cut, refined our fasta to make our OTU centroids.
# THEN, we went back to our original QC'd total fasta file and will throw it all against these nicely defined seeds.
# Anything that doesn't match, we won't keep.
!awk -F"_" \
'BEGIN{OFS=";"}{ if ( substr($1,0,1) == ">"){ print $0,"barcodelabel=",$1 } else { print $0 } }' \
data/finalQC_Pool1_ITS_nound.fasta | \
sed 's/;>//' > data/finalQC_Pool1_ITS_usearchfmt.fasta

In [ ]:
!head data/finalQC_Pool1_ITS_usearchfmt.fasta

>13C.NTH.D7.R4.Frac11_0;barcodelabel=13C.NTH.D7.R4.Frac11
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGGCTCTGGC
>13C.PTH.D30.R4.Frac12_1;barcodelabel=13C.PTH.D30.R4.Frac12
TTACGAGTTGGCCTTTGGCCCGGCTTTTGCCAACCCTGTGAACTTATATTACTGCCTTGGCGGCTTGCGTCTAACGCGCCGCCAGCGGCCTAAACGCACCCTGCTCAGACTAATCTGACTAATGATAACGAGTAAAAACTTTTGGCAACGGATCTCTTGGCTCTGGC
>13C.PTH.D14.R4.Frac22_2;barcodelabel=13C.PTH.D14.R4.Frac22
TTAAACTCATCTCAATTTATTACCTGTAACCCGCCTTAAACCGGGCACACCGGGATCGGGCCTGATAGACTTCACGGTTTGTCGGGTGGCAACCCAATAACCAAAACCTTTTTTAACTTTGTCCATTTCCCGGCTTCCGAAAGGTTGTCGGTTTTCCATCAAACACCATTACCGGTGGATGACTCGGCTCAGAC
>13X.NTH.D3.R4.Frac25_3;barcodelabel=13X.NTH.D3.R4.Frac25
TTATCGAGTTTTGACTGGGTTGTAGCTGGCCTTCCGAGGCATGTGCACGCCCTGCTCATCCACTCTACACCTGTGCACTTACTGTGGGTATCAGATCGTGAAGCGTGCTCTTTTACCGGAGCTTGTGAAGCGTGTCTGTGCCTGCGTTTATCACAAACACTATAAAGTATCAGAATGTGTATTACGATGTAACGCATCTATATACAACTTTCAGCAACGGATCTCTTGGCTCT

In [ ]:
# This is where the actual OTUs are being assigned. We choose 97% sequence ID threshold here.
# This might take a while - like 5 minutes
# Depending ont he clustering algorithm, like pairwise... it would take, like, days on the same number of processors.
# This is why usearch (centroid-based) is so much better
# But is it more biologically relevant? ... maybe, maybe not.
# Edgar is showing it's not that bad.

# We take our total QC data (modified above to have the sample ID extracted)
# We compare it to the otusn.pick.fasta database we made above
# We produce a readmap.uc file which tells us how the reads from our finalQC file map to the otusn seed database.

!usearch -usearch_global data/finalQC_Pool1_ITS_usearchfmt.fasta \
-db data/otusn_Pool1_ITS.fasta \
-strand plus -id 0.97 \
-uc data/readmap_Pool1_ITS.uc \
-threads 15

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:00  19Mb Reading data/otusn_Pool1_ITS.fasta, 3.7Mb
00:00  23Mb 23192 (23.2k) seqs, min 50, avg 146, max 443nt
00:00  23Mb  100.0% Masking
00:00  23Mb  100.0% Word stats
00:00  36Mb  100.0% Building slots
00:00  36Mb  100.0% Build index


In [ ]:
# Makes an OTU table
# It will tell me the OTU ID, and then for all the samples, which OTUs it has sequences from.
!python /opt/bioinfo/edgar_python_scripts/uc2otutab.py data/readmap_Pool1_ITS.uc > data/otu_table_Pool1_ITS.txt

In [ ]:
# Issues with biom table formatting
!if [ -f data/otu_table_Pool1_ITS.biom ]; then rm data/otu_table_Pool1_ITS.biom; fi #This is to mitigate a biom bug
!biom convert -i data/otu_table_Pool1_ITS.txt -o data/otu_table_Pool1_ITS.biom --table-type "otu table"

In [ ]:
# Issues with biom table formatting
!if [ -f data/otu_table_Pool1_ITS_summary.txt ]; then rm data/otu_table_Pool1_ITS_summary.txt; fi #This is to mitigate a biom bug
!biom summarize-table -i data/otu_table_Pool1_ITS.biom -o data/otu_table_Pool1_ITS_summary.txt

In [60]:
# This tells us the overall data info
# Num obs = OTUs
# total count = total seqs
# Chantal had 50% reduction after QC.

!cat data/otu_table_Pool1_ITS_summary.txt



Num samples: 192
Num observations: 23185
Total count: 6582971
Table density (fraction of non-zero values): 0.060
Table md5 (unzipped): 60f55530115a6c2ede5aba02d39b53e0

Counts/sample summary:
 Min: 59.0
 Max: 80171.0
 Median: 34640.500
 Mean: 34286.307
 Std. dev.: 16134.956
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
 13C.PTH.D14.R4.H2O: 59.0
 13C.PTH.D7.R4.H2O: 69.0
 13X.NTH.D3.R4.H2O: 75.0
 13C.PTH.D30.R4.H2O: 120.0
 12C.NTH.D7.R4.H2O: 124.0
 12C.NTH.D30.R4.H2O: 1318.0
 13X.NTH.D7.R4.H2O: 2528.0
 13C.PTH.D7.R4.Frac19: 4540.0
 13C.PTH.D7.R4.Frac18: 5210.0
 13C.PTH.D7.R4.Frac20: 6169.0
 13C.PTH.D7.R4.Frac17: 7548.0
 13X.NTH.D3.R4.Frac18: 10306.0
 13C.PTH.D7.R4.Frac21: 10552.0
 13X.NTH.D3.R4.Frac17: 12404.0
 13C.PTH.D14.R4.Frac18: 13084.0
 12C.NTH.D7.R4.Frac17: 13104.0
 13X.NTH.D3.R4.Frac19: 13381.0
 13X.NTH.D3.R4.Frac14: 13894.0
 13X.NTH.D3.R4.Frac15: 14253.0
 13C.PTH.D30.R4.Frac19: 14442.0
 13C.PTH.D30.R4.Frac18: 14